# Cell与参数

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/zh_cn/advanced/modules/mindspore_layer.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.svg)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/zh_cn/advanced/modules/mindspore_layer.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.svg)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/advanced/modules/layer.ipynb)

Cell作为神经网络构造的基础单元，与神经网络层(Layer)的概念相对应，对Tensor计算操作的抽象封装，能够更准确清晰地对神经网络结构进行表示。除了基础的Tensor计算流程定义外，神经网络层还包含了参数管理、状态管理等功能。而参数(Parameter)是神经网络训练的核心，通常作为神经网络层的内部成员变量。本节我们将系统介绍参数、神经网络层以及其相关使用方法。

## Parameter

参数(Parameter)是一类特殊的Tensor，是指在模型训练过程中可以对其值进行更新的变量。MindSpore提供`mindspore.Parameter`类进行Parameter的构造。为了对不同用途的Parameter进行区分，下面对两种不同类别的Parameter进行定义：

- 可训练参数。在模型训练过程中根据反向传播算法求得梯度后进行更新的Tensor，此时需要将`required_grad`设置为`True`。
- 不可训练参数。不参与反向传播，但需要更新值的Tensor（如BatchNorm中的`mean`和`var`变量），此时需要将`requires_grad`设置为`False`。

> Parameter默认设置`required_grad=True`。

下面我们构造一个简单的全连接层：

In [1]:
import numpy as np
import mindspore
from mindspore import nn
from mindspore import ops
from mindspore import Tensor, Parameter

class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.w = Parameter(Tensor(np.random.randn(5, 3), mindspore.float32), name='w') # weight
        self.b = Parameter(Tensor(np.random.randn(3,), mindspore.float32), name='b') # bias

    def construct(self, x):
        z = ops.matmul(x, self.w) + self.b
        return z

net = Network()

在`Cell`的`__init__`方法中，我们定义了`w`和`b`两个Parameter，并配置`name`进行命名空间管理。在`construct`方法中使用`self.attr`直接调用参与Tensor运算。

### 获取Parameter

在使用Cell+Parameter构造神经网络层后，我们可以使用多种方法来获取Cell管理的Parameter。

#### 获取单个参数

单独获取某个特定参数，直接调用Python类的成员变量即可。

In [2]:
print(net.b.asnumpy())

[-1.2192779  -0.36789745  0.0946381 ]


#### 获取可训练参数

可使用`Cell.trainable_params`方法获取可训练参数，通常在配置优化器时需调用此接口。

In [3]:
print(net.trainable_params())

[Parameter (name=w, shape=(5, 3), dtype=Float32, requires_grad=True), Parameter (name=b, shape=(3,), dtype=Float32, requires_grad=True)]


#### 获取所有参数

使用`Cell.get_parameters()`方法可获取所有参数，此时会返回一个Python迭代器。

In [4]:
print(type(net.get_parameters()))

<class 'generator'>


或者可以调用`Cell.parameters_and_names`返回参数名称及参数。

In [5]:
for name, param in net.parameters_and_names():
    print(f"{name}:\n{param.asnumpy()}")

w:
[[ 4.15680408e-02 -1.20311625e-01  5.02573885e-02]
 [ 1.22175144e-04 -1.34980649e-01  1.17642188e+00]
 [ 7.57667869e-02 -1.74758151e-01 -5.19092619e-01]
 [-1.67846107e+00  3.27240258e-01 -2.06452996e-01]
 [ 5.72323874e-02 -8.27963874e-02  5.94243526e-01]]
b:
[-1.2192779  -0.36789745  0.0946381 ]


### 修改Parameter

#### 直接修改参数值

Parameter是一种特殊的Tensor，因此可以使用Tensor索引修改的方式对其值进行修改。

In [6]:
net.b[0] = 1.
print(net.b.asnumpy())

[ 1.         -0.36789745  0.0946381 ]


#### 覆盖修改参数值

可调用`Parameter.set_data`方法，使用相同Shape的Tensor对Parameter进行覆盖。该方法常用于使用Initializer进行[Cell遍历初始化](https://www.mindspore.cn/docs/zh-CN/master/model_train/custom_program/initializer.html#cell%E9%81%8D%E5%8E%86%E5%88%9D%E5%A7%8B%E5%8C%96)。

In [7]:
net.b.set_data(Tensor([3, 4, 5]))
print(net.b.asnumpy())

[3. 4. 5.]


#### 运行时修改参数值

参数的主要作用为模型训练时对其值进行更新，在反向传播获得梯度后，或不可训练参数需要进行更新，都涉及到运行时参数修改。由于MindSpore的[使用静态图加速](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/accelerate_with_static_graph.html)编译设计，此时需要使用`mindspore.ops.assign`接口对参数进行赋值。该方法常用于[自定义优化器](https://www.mindspore.cn/docs/zh-CN/master/model_train/custom_program/optimizer.html#%E8%87%AA%E5%AE%9A%E4%B9%89%E4%BC%98%E5%8C%96%E5%99%A8)场景。下面是一个简单的运行时修改参数值样例：

In [8]:
import mindspore as ms

@ms.jit
def modify_parameter():
    b_hat = ms.Tensor([7, 8, 9])
    ops.assign(net.b, b_hat)
    return True

modify_parameter()
print(net.b.asnumpy())

[7. 8. 9.]


### Parameter Tuple

变量元组ParameterTuple，用于保存多个Parameter，继承于元组tuple，提供克隆功能。

如下示例提供ParameterTuple创建方法：

In [10]:
from mindspore.common.initializer import initializer
from mindspore import ParameterTuple
# 创建
x = Parameter(default_input=ms.Tensor(np.arange(2 * 3).reshape((2, 3))), name="x")
y = Parameter(default_input=initializer('ones', [1, 2, 3], ms.float32), name='y')
z = Parameter(default_input=2.0, name='z')
params = ParameterTuple((x, y, z))

# 从params克隆并修改名称为"params_copy"
params_copy = params.clone("params_copy")

print(params)
print(params_copy)

(Parameter (name=x, shape=(2, 3), dtype=Int64, requires_grad=True), Parameter (name=y, shape=(1, 2, 3), dtype=Float32, requires_grad=True), Parameter (name=z, shape=(), dtype=Float32, requires_grad=True))
(Parameter (name=params_copy.x, shape=(2, 3), dtype=Int64, requires_grad=True), Parameter (name=params_copy.y, shape=(1, 2, 3), dtype=Float32, requires_grad=True), Parameter (name=params_copy.z, shape=(), dtype=Float32, requires_grad=True))


## Cell训练状态转换

神经网络中的部分Tensor操作在训练和推理时的表现并不相同，如`nn.Dropout`在训练时进行随机丢弃，但在推理时则不丢弃，`nn.BatchNorm`在训练时需要更新`mean`和`var`两个变量，在推理时则固定其值不变。因此我们可以通过`Cell.set_train`接口来设置神经网络的状态。

`set_train(True)`时，神经网络状态为`train`, `set_train`接口默认值为`True`:

In [11]:
net.set_train()
print(net.phase)

train


`set_train(False)`时，神经网络状态为`predict`：

In [12]:
net.set_train(False)
print(net.phase)

predict
